In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.text import *
from fastai.vision import *
# import textwrap

In [ ]:
PATH = Path('data/IAM_handwriting')

In [ ]:
device = torch.device('cpu')#torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Load Data

## From CSV

In [ ]:
fname = 'edited_line.csv'  #'edited_word.csv'
CSV = PATH/fname
FOLDER = 'lines'

df = pd.read_csv(CSV)
len(df)

In [ ]:
df.head()

## Convert pg.csv from ids to text

In [ ]:
df = pd.read_csv(PATH/'pg.csv')
len(df)

In [ ]:
itos = pickle.load(open(PATH/'itos.pkl', 'rb'))
len(itos)

In [ ]:
def convert(x):
    return ''.join([itos[int(c)] for c in x[:-2].split(' ')])

df['label'] = df.apply(lambda row: convert(row.char_ids), axis=1)

## From Source

In [ ]:
#words
df = pd.read_csv(f'{PATH}/ascii/words.txt', names=['filename','result','word'], escapechar='\\',
                 delim_whitespace=True, skiprows=18, header=None, usecols=[0,1,8])
len(df)

In [ ]:
#lines
df = pd.read_csv(f'{PATH}/ascii/lines.txt', names=['filename','result','text'], escapechar='\\',
                 delim_whitespace=True, skiprows=23, header=None, usecols=[0,1,8])
df.apply(lambda row: row.text.replace('|', ' '), axis=1)
len(df)

### via manually created DF

In [ ]:
maxTextLen = 32
samples = []
chars = set()

f=open(f'{PATH}/ascii/words.txt')
for line in f:
    # ignore comment line
    if not line or line[0]=='#':
        continue

    lineSplit = line.strip().split(' ')
    assert len(lineSplit) >= 9

    fileName = lineSplit[0]

    # GT text are columns starting at 9
    gtText = ''.join(lineSplit[8:])[:maxTextLen]
    char_len = len(gtText)
    chars = chars.union(set(list(gtText)))

    # put sample into list
    samples.append([fileName, gtText, char_len])
    
samples = np.stack(samples)
df = pd.DataFrame(samples, columns=['filename', 'word', 'char_len'], )
del samples

In [ ]:
df['char_len'] = df.char_len.astype('int32')
df = df.loc[df['char_len'] > 3]
df = df.loc[df['char_len'] < 20]
df.head()

# Edit Downloaded Data

In [ ]:
dl_path = Path('data/handwriting_images')
# b = pd.read_csv(dl_path/'ewd.csv')
# a = pd.read_csv(dl_path/'downloads.csv')
# img_path = dl_path/'downloads'

In [ ]:
df = pd.concat([a,b], ignore_index=True)

In [ ]:
num_lines = df.text.apply(lambda x: len(x.split('\n')))
df['num_lines'] = num_lines
df.head()

In [ ]:
df.sort_values('num_lines', inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.rename({'text': 'label'}, axis='columns')

In [ ]:
df.to_csv(dl_path/'dl.csv', index=False)

In [ ]:
len(df)

In [ ]:
i = 109
row = df.iloc[i]
print(row.text)
print(row.filename)

In [ ]:
t1 = '''main task is to give a formal proof that the
program he proposes meets the equally formal
functional specification. While designing proofs
and programs hand in hand, the student gets
ample opportunity to perfect his manipulative
agility with the predicate calculus. Finally, in
order to drive home the message that this'''

t2 = '''introductory programming course is primarily
a course in formal mathematics, we see to it
that the programming language in question
has not been implemented on campus so that
students are protected from the temptation
to test their programs. And this concludes
the sketch of my proposal for an introductory
programming course for freshmen.'''

In [ ]:
row.text = t1
row.filename = row.filename[:-4]+'a.png'

line = DataFrame({"filename": row.filename[:-5]+'b.png', "text": t2}, index=[i+0.1])
df = df.append(line, ignore_index=False)

In [ ]:
df = df.sort_index().reset_index(drop=True)

In [ ]:
df = pd.read_csv(dl_path/'downloads.csv')

In [ ]:
for i in range(1,11):
    df = pd.read_csv(dl_path/'downloads.csv')
    df['filename'] = df['filename'].apply(lambda x: f"v{i}_{x}")
    df.to_csv(dl_path/f"v{i}.csv", index=False)

In [ ]:
a = pd.read_csv(dl_path/'v1.csv')
b = pd.read_csv(dl_path/'v2.csv')
c = pd.read_csv(dl_path/'v3.csv')
d = pd.read_csv(dl_path/'v4.csv')
e = pd.read_csv(dl_path/'v5.csv')
f = pd.read_csv(dl_path/'v6.csv')
g = pd.read_csv(dl_path/'v7.csv')
h = pd.read_csv(dl_path/'v8.csv')
i = pd.read_csv(dl_path/'v9.csv')
j = pd.read_csv(dl_path/'v10.csv')

In [ ]:
ndf = pd.concat([df,a,b,c,d,e,f,g,h,i,j], ignore_index=True)
ndf.sort_values('num_lines', inplace=True)
ndf.reset_index(inplace=True, drop=True)

In [ ]:
ndf.to_csv(dl_path/'downloaded_images.csv', index=False)

In [ ]:
len(ndf)

# Remove test data

In [ ]:
test = df.sample(15, random_state=42)
test

In [ ]:
# a01-122
# a03-014
# b04-162
# b06-071
# f04-093
# g06-042j
# j01-045
# l01-030
# l01-173
# l03-004
# p02-105
# p03-040
# r02-081
# r03-096
# r06-115

In [ ]:
test.to_csv(PATH/'test_pg.csv', columns=['filename', 'label'], index=False)

In [ ]:
df = df.drop(test.index)
len(df)

In [ ]:
df.to_csv(PATH/'edited_pg.csv', columns=['filename', 'label'], index=False)

# Edit Data

## Filter df

In [ ]:
def cleanup(x):
    x = re.sub(r" \'(\w+)", lambda x: x.group(0)[1:], x)     # match all space and apostrophe letters
    x = re.sub(r" [.,?!:;()]", lambda x: x.group(0)[1:], x)  # match all space and punctuation
    return x

df['text'] = df.apply(lambda row: cleanup(row.text), axis=1)

In [ ]:
res = df[df.char_len == 1]
res = res[res['word'].str.contains("[.,\-\"']")]
res.head()

In [ ]:
df.drop(res.index, inplace=True)
len(df)

In [ ]:
def cleanup(x): return x.replace('""', '" "')

In [ ]:
def respace_quotes(m):
    m = m.group(0)   # entire matched string
    m = m.replace('"','" ')
#     m = m.replace(' "',' " ')
    return m

# def clean_quotes(x): return re.sub(r'\"(.+?)\"', despace_quotes, x)
def clean_quotes(x): return re.sub(r'\"\S', respace_quotes, x)

In [ ]:
df['text'] = df.apply(lambda row: clean_quotes(row.text), axis=1)

In [ ]:
# df = pd.read_csv(PATH/'edited_line.csv')

## Fix Errors

In [ ]:
err = df[df.result == 'err']
len(err)

In [ ]:
itr = iter(err.index.values)

In [ ]:
import pyperclip

In [ ]:
i = 348 #next(itr)
row = df.iloc[i]
print(i)#, row.result)
im_path = PATH/'words'/f'{row.filename}.png'
im = PIL.Image.open(im_path)
fig,ax = plt.subplots(figsize=(16,3))
# row.result = 'ok'
ax.imshow(im) 
# pyperclip.copy(row.text)
# row.text

In [ ]:
row.result = 'err'

In [ ]:
row.text = 'went to her fiance*? 2\'s house and'

### crop image

In [ ]:
w,h = im.size
nim = im.crop((0,0,w,130))

In [ ]:
nim.save(im_path)
nim

### append multiple rows

In [ ]:
st="a06-095-05-04 ok 182 1331 1586 153 115 ATI The\na06-095-05-05 ok 182 1536 1571 490 175 NN Community\na06-095-06-00 ok 187 305 1755 81 145 ( (\na06-095-06-01 ok 187 379 1768 380 125 JJ Common\na06-095-06-02 ok 187 829 1754 370 123 NN Market\na06-095-06-03 ok 187 1173 1751 60 155 ) )\na06-095-06-04 ok 187 1266 1762 203 119 HVZ has\na06-095-06-05 ok 187 1524 1758 486 168 VBN imparted\na06-095-06-06 ok 187 2043 1800 128 83 AT an\na06-095-07-00 ok 183 309 1941 442 157 NN impetus\na06-095-07-01 ok 183 816 1958 261 118 CC and\na06-095-07-02 ok 183 1121 1988 152 85 AT an\na06-095-07-03 ok 183 1350 1971 493 89 JJ economic\na06-095-07-04 ok 183 1895 1943 301 173 NN growth\na06-095-08-00 ok 183 324 2127 102 91 IN to\na06-095-08-01 ok 183 516 2125 170 104 ATI The\na06-095-08-02 ok 183 743 2120 145 127 CD Six\na06-095-08-03 ok 183 930 2217 10 21 . .\na06-095-08-04 ok 183 1027 2110 320 157 IN Above\na06-095-08-05 ok 183 1387 2113 143 115 ABN all\na06-095-08-06 ok 183 1496 2216 46 72 , ,\na06-095-08-07 ok 183 1602 2112 116 117 PP3 it\na06-095-08-08 ok 183 1731 2128 95 105 BEZ is\na06-095-08-09 ok 183 1861 2167 122 60 AT an\na06-095-08-10 ok 183 2027 2128 220 124 NN idea\na06-095-09-00 ok 182 323 2290 287 121 WDT which\na06-095-09-01 ok 182 678 2295 172 117 HVZ has\na06-095-09-02 ok 182 905 2310 372 179 VBN gripped\na06-095-09-03 ok 182 1315 2294 283 128 NNS$ men's\na06-095-09-04 ok 182 1654 2318 322 105 NNS minds\na06-095-09-05 ok 182 1962 2404 23 43 , ,\n"

In [ ]:
with open("tmp.txt", "w") as text_file:
    text_file.write(st)

f=open('tmp.txt')
samples = []
for line in f:
    # ignore comment line
    if not line or line[0]=='#':
        continue

    lineSplit = line.strip().split(' ')
    assert len(lineSplit) >= 9

    fileName = lineSplit[0]
    res = lineSplit[1]

    # GT text are columns starting at 9
    gtText = ''.join(lineSplit[8:])[:32]

    # put sample into list
    samples.append([fileName, res, gtText])
    
samples = np.stack(samples)
tmp_df = pd.DataFrame(samples, columns=['filename', 'result', 'word'], )
del samples

In [ ]:
tmp_itr = iter(tmp_df[tmp_df.result == 'err'].index.values)

i = next(tmp_itr)
print(i);row = tmp_df.iloc[i]
im_path = PATH/'words'/f'{row.filename}.png'
im = PIL.Image.open(im_path)
fig,ax = plt.subplots(figsize=(3,3))
ax.imshow(im);ax.set_title(row.word)

In [ ]:
row.result = 'ok'

In [ ]:
row.text = 'Leftwing'

In [ ]:
df = df.append(tmp_df[tmp_df.result != 'err'], ignore_index=True, sort=False)
len(df)

### append single

In [ ]:
fname = 'c04-134-01-06'
im_path = PATH/'words'/f'{fname}.png'
im = PIL.Image.open(im_path)
im

In [ ]:
df = df.append({'filename':fname, 'result':'ok', 'word':'Greensleeves'}, ignore_index=True)
len(df)

### paste images

In [ ]:
pw,ph = pim.size
pw,ph

In [ ]:
w,h = im.size
w,h

In [ ]:
nim = PIL.Image.new('RGB', (w+pw,ph), color=(255,255,255))

In [ ]:
nim.paste(pim, (0,0))

In [ ]:
nim.paste(im, (pw,ph-h))

In [ ]:
nim.save(im_path)
nim

### save csv

In [ ]:
df.to_csv(PATH/'edited_line.csv', columns=['filename', 'result', 'text'], index=False)

## Add char_lens

In [ ]:
# calculate character lengths
lgts = df.label.apply(len)  
df['char_len'] = lgts.astype('int32')

## Add num_lines

In [ ]:
fname = 'edited_pg.csv'  #'edited_word.csv'
CSV = PATH/fname
FOLDER = 'paragraphs'

df = pd.read_csv(CSV)
len(df)

In [ ]:
num_lines = df.label.apply(lambda x: len(x.split('\n')))
df['num_lines'] = num_lines
df.head()

In [ ]:
df.to_csv(CSV, index=False)

In [ ]:
six_lines = df[df.num_lines>=6]
six_lines.to_csv(PATH/'edited_cat6up.csv', index=False)

# Helpers

In [ ]:
def standardize_imgs(imgs, baseheight):
    resized_imgs = []
    for img in imgs:
        hpercent = (baseheight / float(img.size[1]))
        wsize = int((float(img.size[0]) * float(hpercent)))
        img = img.resize((wsize, baseheight), PIL.Image.ANTIALIAS)
        resized_imgs.append(img)
    return resized_imgs

In [ ]:
def resize_max(im, size=1000):
    "Resize an image so that the largest dimension is of specified size"
    r,c = im.size
    ratio = size/max(r,c)
    return im.resize((int(r*ratio), int(c*ratio)), PIL.Image.ANTIALIAS)

In [ ]:
def variable_padding(im, top_left=False, size=None):
    padLeft = random.randint(30,100)
    padTop = random.randint(30,100)
        
    new_im = PIL.Image.new('RGB', (sz, sz), color=(255,255,255))  # new white image

    new_im.paste(im, box=(padLeft, padTop))
    return new_im

In [ ]:
def square_max(im, top_left=False, size=None):
    '''
    Add whitespace to square an image by its largest dimension or specified size.
    Args:
        top_left: image is aligned with the top_left corner
        size: size of final squared image.  If left blank size = largest dimension
    '''
    
    r,c = im.size
    if size is not None and size > max(r,c):
        sz = size
    else:
        sz = max(r,c)
        
    new_im = PIL.Image.new('RGB', (sz, sz), color=(255,255,255))  # new white image
    
    # box logic
    if top_left:
        box = (0,0)
    else:
        if sz == r:
            box = (0,random.randint(0,sz-c)) 
        elif sz == c:
            box = (random.randint(0,sz-r),0)
        else:
            box = (random.randint(20,sz-r),random.randint(20,sz-c))
            
    new_im.paste(im, box=box)
    return new_im

In [ ]:
def resize_dir(fn, src, targ=None):
    if targ is None: targ = src
    dirs = os.listdir(src)
    for item in progress_bar(dirs):
        if os.path.isdir(src/item): continue     # skip if src dir
#         if os.path.isfile(targ/item): continue   # skip if file exists in targ dir
        im = PIL.Image.open(src/item)
        rsz = fn(im)
        rsz.save(targ/item)
        rsz.close()
        im.close()

In [ ]:
def resize_to_square(src_dir, targ_dir, size):
    "Resize and square all images in src_dir and save in targ_dir"
    resize_dir(partial(resize_max, size=size-50), src_dir, targ_dir)
    resize_dir(partial(square_max, size=size), targ_dir)

In [ ]:
def show_sample(df, path, row=2, col=2, show_files=False):
    fig, axes = plt.subplots(row,col, figsize=(20, 10))
    samp = df.sample(row*col).values #=> outputs as an array [[filename, labels]]
    for i,ax in enumerate(axes.flat):
        row = samp[i]
        ax.imshow(PIL.Image.open(path/row[0]))
        title = row[1]+f"\n{row[2]}" if show_files else row[1]
        ax.set_title(title)

#     plt.tight_layout(pad=0.2)

# Synthesize Lines

In [ ]:
def create_img(imgs, targ_path, num_lines, max_size, pad=30):
    w = 1
    h = num_lines
        
    widths, heights = zip(*(i.size for i in imgs))
    
    median_height = int(np.median(heights))
    stzd_imgs = standardize_imgs(imgs, median_height)
    lines = [stzd_imgs[i:i + w] for i in range(0, len(stzd_imgs), w)]
    
    total_width = max([np.sum([word.size[0] for word in line]) for line in lines]) + (pad*(w+1))   
    total_height = (median_height * h) + (pad*(h+1)) #sum(heights)

    new_im = PIL.Image.new('RGB', (total_width, total_height), color=(255,255,255))

    y_offset = pad
    x_offset = pad
    
    for line in lines:
        x_offset = pad
        for word in line:
            new_im.paste(word, (x_offset,y_offset))
            x_offset += word.size[0] + pad
        y_offset += median_height + pad
    
    if max_size: 
        resize_max(new_im, max_size).save(targ_path)
    else:
        new_im.save(targ_path/fname)

In [ ]:
# number of words/image
def create_synth_data(df, num, num_lines, src_path, targ_path, max_size=1000, offset=0):
    d={}
    for i in progress_bar(range(num)):
        samp = df.sample(num_lines)
        files = list(map(lambda x: x+'.png', samp.filename.values))
        imgs  = [PIL.Image.open(src_path/f) for f in files]
        
        # split into rows with \n
        label = '\n'.join([' '.join(row) for row in np.array_split(samp.text.values, num_lines)])
#         label = ' '.join(samp.text.values)

        fname = str(num_lines)+'_'+'{:04d}'.format(i+offset)+'.png'
        create_img(imgs, targ_path/fname, num_lines, max_size)
        [f.close() for f in imgs]
        d[fname] = label
    return d

In [ ]:
src_path = PATH/'lines'
synth_path = PATH/'edited_cat_lines'

!rm -rf {synth_path}
os.makedirs(synth_path, exist_ok=True)

In [ ]:
#single
num_lines = 3
d = create_synth_data(df, 10, num_lines, src_path, synth_path)

synth = pd.DataFrame({'filename': list(d.keys()), 'labels': list(d.values())})
synth.head()

In [ ]:
show_sample(synth, synth_path)

In [ ]:
#multi
for num_lines in progress_bar(range(3,15)):
    d = create_synth_data(df, 1000, num_lines, src_path, synth_path)
    synth = pd.DataFrame({'filename': list(d.keys()), 'label': list(d.values())})
    
    CSV = str(synth_path)+'_'+str(num_lines)+'.csv'
    synth.to_csv(CSV, columns=['filename', 'label'], index=False)

In [ ]:
a = pd.read_csv(PATH/'edited_cat_lines_3.csv')
b = pd.read_csv(PATH/'edited_cat_lines_4.csv')
c = pd.read_csv(PATH/'edited_cat_lines_5.csv')
d = pd.read_csv(PATH/'edited_cat_lines_6.csv')
e = pd.read_csv(PATH/'edited_cat_lines_7.csv')
f = pd.read_csv(PATH/'edited_cat_lines_8.csv')
g = pd.read_csv(PATH/'edited_cat_lines_9.csv')
h = pd.read_csv(PATH/'edited_cat_lines_10.csv')
i = pd.read_csv(PATH/'edited_cat_lines_11.csv')
j = pd.read_csv(PATH/'edited_cat_lines_12.csv')
k = pd.read_csv(PATH/'edited_cat_lines_13.csv')
l = pd.read_csv(PATH/'edited_cat_lines_14.csv')

In [ ]:
new = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l], ignore_index=True)
len(new)

In [ ]:
CSV = str(synth_path) + '.csv'
new.to_csv(CSV, columns=['filename', 'label'], index=False)

In [ ]:
show_sample(new, synth_path)

# Synthesize Words

## num words / line

In [ ]:
def create_img(src_path, targ_path, files, fname, sz=None, pad=30):
    if sz==None: sz=(1, len(files))  #(w,h)
    w = sz[1]
    h = sz[0]
        
    imgs = [ PIL.Image.open(src_path/f) for f in files ]
    widths, heights = zip(*(i.size for i in imgs))
    
    median_height = int(np.median(heights))
    stzd_imgs = standardize_imgs(imgs, median_height)
    lines = [stzd_imgs[i:i + w] for i in range(0, len(stzd_imgs), w)]
    
    total_width = max([np.sum([word.size[0] for word in line]) for line in lines]) + (pad*(w+1))   
    total_height = (median_height * h) + (pad*(h+1)) #sum(heights)

    new_im = PIL.Image.new('RGB', (total_width, total_height), color=(255,255,255))

    y_offset = pad
    x_offset = pad
    
    for line in lines:
        x_offset = pad
        for word in line:
            new_im.paste(word, (x_offset,y_offset))
            x_offset += word.size[0] + pad
        y_offset += median_height + pad
        
    new_im.save(targ_path/fname)

In [ ]:
# number of words/image
def create_synth_data(df, src_path, targ_path, num, sz, offset=0, randomize=False, pre=''):
    d=[]
    for i in progress_bar(range(num)):
        if randomize:
            r = random.randint(1,sz[0])
            c = random.randint(1,sz[1])
        else:
            r,c = sz
            
        num_samp = np.product((r,c))
        res = df.sample(num_samp)
        files = list(map(lambda x: x+'.png', res.filename.values))
        
        # split into rows with \n
        label = '\n'.join([' '.join(row) for row in np.array_split(res.word.values, r)])
#         label = ' '.join(res.word.values)
        
        fname = pre+'{:04d}'.format(i+offset)+'.png'
        create_img(src_path, targ_path, files, fname, (r,c))
        row = {'filename': fname, 'label': label, 'source': files}
        d.append(row)
    return d

In [ ]:
src_path = PATH/'words'
synth_path = PATH/'edited_sm_synth'
!rm -rf {synth_path}

os.makedirs(synth_path, exist_ok=True)

In [ ]:
d = create_synth_data(df, src_path, synth_path, 50000, (4,3), randomize=True, pre='sm_')
len(d)

In [ ]:
synth = pd.DataFrame(d)
synth.head()

In [ ]:
# resize_dir(partial(resize_max, size=512), src_path)
# resize_dir(partial(square_max, size=1000), synth_path)

In [ ]:
show_sample(synth, synth_path, show_files=True)

In [ ]:
CSV = str(synth_path) + '.csv'
synth.to_csv(CSV, columns=['filename', 'label'], index=False)

## size of image

In [ ]:
synth_path = PATH/'large_synth_words_test'
!rm -rf {synth_path}

os.makedirs(synth_path, exist_ok=True)

In [ ]:
def create_img(sz, fname, pad=30, median_height=None):
    # TODO: randomize padding
    new_im = Image.new('RGB', (sz,sz), color=(255,255,255))
    
    res   = df.sample(50)
    files = list(map(lambda x: x+'.png', res.filename.values))
    lbls  = res.word.values.tolist()
    imgs  = [ PIL.Image.open(PATH/'words'/f) for f in files ]
    
    if median_height is None:
        w, h  = zip(*(i.size for i in imgs))
        # standardize heights and sort longest to shortest words
        median_height = int(np.median(h))        # TODO: randomize this between mean/std
        
    stzd_imgs = standardize_imgs(imgs, median_height)
    
    #loop through standardized images and find the next image which satisfies the condition
    labels = []
    y_offset = pad
    while y_offset+median_height+pad < sz:        
        x_offset = pad
        
        gen = (i for i,x in enumerate(stzd_imgs) if x.size[0]+x_offset+pad <= sz)
        lines = []
        for idx in gen:
            word = stzd_imgs.pop(idx)            
            lines.append(lbls.pop(idx))
            new_im.paste(word, (x_offset,y_offset))
            x_offset += word.size[0] + pad
        y_offset += median_height+pad
        labels.append(' '.join(lines))

    new_im.save(synth_path/fname)    
    return '\n'.join(labels)

In [ ]:
# size of image
def create_synth_data(qty, sz, fname_offset=0):
    d={}
    for i in tqdm(range(qty)):
        fname = '{:04d}'.format(i+fname_offset)+'.png'
        p = random.randint(10,20)
        h = random.randint(25,35)
        d[fname] = create_img(sz, fname, pad=p, median_height=h)
    return d

d = create_synth_data(20000, 512)
len(d)
# ~25min to create 5000 1000x1000 images

In [ ]:
synth = pd.DataFrame({'filename': list(d.keys()), 'labels': list(d.values())})
synth.head()

In [ ]:
show_sample(synth, synth_path)

# Numericalize

## Chars

In [ ]:
# same as used in single word / multi-word
itos = pickle.load(open(PATH/'char_itos.pkl', 'rb'))
len(itos)

In [ ]:
joined_labels = list(synth.labels) #list(map(lambda x: ' '.join(x), labels))

stoi = collections.defaultdict(lambda: 82, {v:k for k,v in enumerate(itos)})
ids = np.array([np.array([stoi[letter] for letter in word] + [3]) for word in joined_labels])

# convert to strings (as labels)
str_ids = np.array([' '.join(str(l) for l in w) for w in ids]).reshape(-1,1)

In [ ]:
synth['char_ids'] = str_ids
synth.head()

In [ ]:
CSV = str(synth_path) + '.csv'
synth.to_csv(CSV, columns=['filename', 'char_ids'], index=False)

In [ ]:
# # multi-line
# CSV = str(targ_path)+'_'+str(num_lines)+'.csv'
# synth.to_csv(CSV, columns=['filename', 'char_ids'], index=False)

### Add to existing CSV

In [ ]:
CSV = PATH/'large_synth_words_10000.csv'
csv = pd.read_csv(CSV)
len(csv)

In [ ]:
# CSV = PATH/'synth_words_50000.csv'
CSV = PATH/'large_synth_words_50000.csv'

new = pd.concat([csv, synth[['filename', 'char_ids']]], ignore_index=True)
new.to_csv(CSV, columns=['filename', 'char_ids'], index=False)

In [ ]:
len(new)

## Concatenate CSVs

In [ ]:
a = pd.read_csv(PATH/'cat_lines_11.csv')
b = pd.read_csv(PATH/'cat_lines_12.csv')
c = pd.read_csv(PATH/'cat_lines_13.csv')
d = pd.read_csv(PATH/'cat_lines_14.csv')

In [ ]:
new = pd.concat([a,b,c,d], ignore_index=True)
len(new)

In [ ]:
new.to_csv(PATH/'cat_lines_11-14.csv', index=False)

In [ ]:
def to_string(row):
    return ''.join([itos[int(c)] for c in row.split(' ')])


fig, axes = plt.subplots(2,2, figsize=(20, 20))
for i,ax in enumerate(axes.flat):
    row = a.iloc[i]
    im = Image.open(targ_path/row.filename)
    ax.imshow(im)
    label = to_string(row.char_ids)
#     label = '\n'.join(textwrap.wrap(row.labels, 70))
    ax.set_title(label)

plt.tight_layout(pad=0.2)

## Words

In [ ]:
itos = pickle.load(open(PATH/'synth_word_itos.pkl', 'rb'))
len(itos)

In [ ]:
joined_labels = list(synth.labels)

stoi = collections.defaultdict(lambda: 2, {v:k for k,v in enumerate(itos)})
ids = np.array([np.array([stoi[word] for word in line.split(' ')]+[3]) for line in joined_labels])

# convert to strings (as labels)
str_ids = np.array([' '.join(str(l) for l in w) for w in ids]).reshape(-1,1)

In [ ]:
synth['word_ids'] = str_ids
synth.head()

### Add to existing CSV

In [ ]:
CSV = PATH/'large_synth_word_ids_10000.csv'
csv = pd.read_csv(CSV)

In [ ]:
# CSV = PATH/'synth_words_50000.csv'
CSV = PATH/'large_synth_word_ids_50000.csv'

new = pd.concat([csv, synth[['filename', 'word_ids']]], ignore_index=True)
new.to_csv(CSV, columns=['filename', 'word_ids'], index=False)

In [ ]:
len(new)

### Modify csv/itos to match previous versions

In [ ]:
itos_old = pickle.load(open(TMP_PATH/'synth_word_itos.pkl', 'rb'))

# same as used in single word / multi-word
itos = pickle.load(open(TMP_PATH/'char_itos.pkl', 'rb'))

In [ ]:
res = [''.join([itos_old[int(c)] for c in line.split(' ')]) for line in csv.char_ids]
csv['words'] = res
csv.head()

In [ ]:
joined_labels = list(csv.words) #list(map(lambda x: ' '.join(x), labels))

stoi = collections.defaultdict(lambda: 2, {v:k for k,v in enumerate(itos)})
ids = np.array([np.array([stoi[letter] for letter in word]+[3]) for word in joined_labels])

In [ ]:
# convert to strings (as labels)
str_ids = np.array([' '.join(str(l) for l in w) for w in ids]).reshape(-1,1)

In [ ]:
csv['char_ids'] = str_ids
csv = csv[['filename', 'char_ids']]
csv.head()

### Verify

In [ ]:
def label_text(pred):
#     ints = to_np(pred).astype(int)
#     ints = np.trim_zeros(ints)   # remove padding (0)
    return ''.join([itos[int(i)] for i in pred])

In [ ]:
fig, axes = plt.subplots(5,1, figsize=(10, 10))
for i,ax in enumerate(axes.flat):
    row = synth.iloc[i]
    im = Image.open(synth_path/row.filename)
    ax.imshow(im)
    ax.set_title(label_text(row.char_ids.split(' ')))
    
plt.tight_layout(pad=0.2)

# Batch Resize

In [ ]:
targ_path = PATH/'test_resize'
os.makedirs(targ_path, exist_ok=True)

In [ ]:
src_path = PATH/'cat_lines'
# targ_path = PATH/'resized_cat_lines'
# os.makedirs(targ_path, exist_ok=True)

In [ ]:
resize_to_square(src_path, targ_path, 1000)

In [ ]:
# resize_dir(partial(resize_max, size=512), src_path)
resize_dir(partial(square_max, size=1000), src_path, targ_path)

# V1 add padding

In [ ]:
src_path = PATH/'fonts'
targ_path = PATH/'fonts_resize'

os.makedirs(targ_path, exist_ok=True)

In [ ]:
def pad_one(fpath, i, targ_path):
    dest = targ_path/fpath.name
    img = PIL.Image.open(fpath)
    bg_color = img.getpixel((0,0))
    bottom_pad = (random.randint(10,100) if img.size[0] > 264 else 0)
    img = PIL.ImageOps.expand(img, border=(random.randint(10,100),random.randint(10,100),0,bottom_pad), fill=bg_color)
    img = resize_max(img)
    img.save(dest)

In [ ]:
il = ImageList.from_folder(src_path)

In [ ]:
parallel(partial(pad_one, targ_path=targ_path), il.items)

# Convert datasets to new itos.pkl

In [ ]:
itos = pickle.load(open(TMP_PATH/'char_itos.pkl', 'rb'))
len(itos)

In [ ]:
voc = pickle.load(open(TMP_PATH/'itos.pkl', 'rb'))
len(voc)

In [ ]:
def convert_char_itos(old_itos, new_itos, path, old_fname, new_fname):
    old_csv = pd.read_csv(path/old_fname)
    # convert to text and remove _eos_ token
    res = [''.join([old_itos[int(c)] for c in line.split(' ')[:-1]]) for line in old_csv.char_ids]    
    # xxunk: 3, xxeos: 2
    stoi = collections.defaultdict(lambda: 3, {v:k for k,v in enumerate(new_itos)})
    ids = np.array([np.array([stoi[letter] for letter in word] + [2]) for word in list(res)])

    # convert to strings (as labels)
    str_ids = np.array([' '.join(str(l) for l in w) for w in ids]).reshape(-1,1)
    old_csv['char_ids'] = str_ids
    old_csv.to_csv(path/new_fname, columns=['filename', 'char_ids'], index=False)

In [ ]:
# old,new = 'small_synth_words.csv','sm_synth.csv'
# old,new = 'multi_synth_words.csv','3x2_synth.csv'
# old,new = 'paragraphs.csv','pg.csv'
# old,new = 'mix_words_dl.csv','full_mix.csv'
# old,new = 'mix_words.csv','mix.csv'
old,new = 'downloaded_images.csv', 'dl.csv'

In [ ]:
convert_char_itos(itos, voc, PATH, old, new)

## Verify

In [ ]:
csv = pd.read_csv(PATH/new)

res = [''.join([voc[int(c)] for c in line.split(' ')[:-1]]) for line in csv.char_ids]
csv['text'] = res
csv.head()

# Create Mix Dataset

## Mix

In [ ]:
dl = pd.read_csv(PATH/'downloaded_images.csv')
sm = pd.read_csv(PATH/'edited_sm_synth.csv')
cat = pd.read_csv(PATH/'edited_cat_lines.csv')
pg = pd.read_csv(PATH/'edited_pg.csv')

In [ ]:
dl.tail(10)

In [ ]:
mix = pd.concat([dl,sm,cat,pg])
len(mix)

In [ ]:
mix

In [ ]:
mix.to_csv(PATH/'mix.csv', index=False)

## <=6 lines:  fonts+cat_lines

In [ ]:
a = pd.read_csv(PATH/'edited_cat6up.csv')
a.head()

In [ ]:
b = pd.read_csv(PATH/'combo6up.csv')
b.head()

In [ ]:
mix = pd.concat([a,b,df], ignore_index=True)
len(mix)

In [ ]:
mix.sort_values('num_lines', inplace=True)
mix.reset_index(inplace=True, drop=True)

In [ ]:
mix.to_csv(PATH/'combo_cat_pg.csv', index=False)
# max:  14 lines, 723 chars

In [ ]:
df = pd.read_csv(PATH/'combo_cat_pg.csv')

# SentencePiece

In [ ]:
import sentencepiece as spm

In [ ]:
df.label.values[:5]

In [ ]:
name = str(PATH/'spm_train')
fname = name + '.txt'

In [ ]:
# puts dataset into format expected by sentencepiece:
# .txt file entries separated by \n
def write_text(texts, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for text in texts:
            f.write(text + "\n")

write_text(df.label.values, fname)

In [ ]:
spm.SentencePieceTrainer.Train(
    f"--unk_id=3 --pad_id=0 --input={fname} --model_prefix={name}_8k --vocab_size=8000 --user_defined_symbols=\n"
)

In [ ]:
sp = spm.SentencePieceProcessor()
sp.Load(name+'_8k.model')
sp.SetEncodeExtraOptions("bos:eos")

In [ ]:
vocab = {i: sp.id_to_piece(i) for i in range(8000)}
vocab

In [ ]:
ids = sp.EncodeAsIds(df.label.values[0])

In [ ]:
sp.DecodeIds(ids+[0,0,0,0,0,0,0,0]) == df.label.values[0]